In [ ]:
import pandas as pd
from utilities.data_manager import DataManager
from utilities.backtest_analysis import BacktestAnalysis
from strategies import envelope as strat

In [ ]:
data = DataManager(name="binance")  # "bitget"

In [ ]:
symbol = "BTC/USDT:USDT"
ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
strategy_params = {
    'average_type': 'SMA',  # 'SMA', 'EMA', 'WMA', 'DCM'
    'average_period': 6,
    'envelopes': [0.07, 0.11, 0.14],
    'stop_loss_pct': 0.3,
    # 'price_jump_pct': 0.3,
    'position_size_percentage': 100,  #  % of the balance spread equally across each envelope
    # 'position_size_fixed_amount': 1000,  # fixed amount spread equally across each envelope
    # 'mode': "long", "short", 'both' (default)
}

# symbol = "ETH/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'DCM',
#     'average_period': 5,
#     'envelopes': [0.07, 0.11, 0.14],
#     'stop_loss_pct': 0.4,
#     'position_size_percentage': 100,
# }

# symbol = "ADA/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'DCM',
#     'average_period': 5,
#     'envelopes': [0.07, 0.11, 0.14, 0.17],
#     'stop_loss_pct': 0.4,
#     'price_jump_pct': 0.3,
#     'position_size_percentage': 100,
# }

# symbol = "AVAX/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'SMA',
#     'average_period': 5,
#     'envelopes': [0.1, 0.15, 0.2, 0.25],
#     'stop_loss_pct': 0.4,
#     'price_jump_pct': 0.3,
#     'position_size_percentage': 100,
# }

# symbol = "SOL/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h", start_date="2021-01-01 00:00:00")
# strategy_params = {
#     'average_type': 'SMA',
#     'average_period': 5,
#     'envelopes': [0.07, 0.11, 0.14, 0.17],
#     'stop_loss_pct': 0.4,
#     # 'price_jump_pct': 0.3,
#     'position_size_percentage': 100,
# }

# symbol = "LINK/USDT:USDT"
# ohlcv = data.load(symbol, timeframe="1h")
# strategy_params = {
#     'average_type': 'SMA',
#     'average_period': 5,
#     'envelopes': [0.1, 0.15, 0.2, 0.25],
#     'stop_loss_pct': 0.4,
#     'price_jump_pct': 0.2,
#     'position_size_percentage': 100,
# }

strategy = strat.Strategy(strategy_params, ohlcv)
strategy.run_backtest(initial_balance=1000, leverage=1, open_fee_rate=0.0002, close_fee_rate=0.0006)

In [ ]:
results = BacktestAnalysis(strategy)
results.print_metrics()
# path = "..."
# results.print_metrics(path)

In [ ]:
strategy.trades_info
# path = symbol.replace('/', '-').replace(':', '-') 
# strategy.save_trades_info(path)

In [ ]:
strategy.equity_record
# strategy.save_equity_record(path)

In [ ]:
results.plot_equity()
# results.plot_equity(path)

results.plot_drawdown()
# results.plot_drawdown(path)

In [ ]:
results.plot_monthly_performance(year="all")
# results.plot_monthly_performance(year="all", path)

In [ ]:
indicators = {
    'average': {
        'color': "white",
        'df': pd.DataFrame({
            'time': strategy.data.index,
            'average': strategy.data['average'],
        }).dropna()        
    } 
}

for i, e in enumerate(strategy_params['envelopes']):
    indicators.update({
        f'band_high_{i+1}' : {
            'color': "red",
            'df': pd.DataFrame({
                'time': strategy.data.index,
                f'band_high_{i+1}': strategy.data[f'band_high_{i+1}'],
            }).dropna()   
        },
        f'band_low_{i+1}' : {
            'color': "green",
            'df': pd.DataFrame({
                'time': strategy.data.index,
                f'band_low_{i+1}': strategy.data[f'band_low_{i+1}'],
            }).dropna()  
        },
    })

results.plot_candlestick(indicators=indicators)